In [1]:
from onmt.train_single import main as single_main
from onmt.new_opts import Opts, PreprocessorOpts
from onmt.preprocess import build_save_dataset, build_save_vocab
import onmt.inputters as inputters

In [2]:
def separate_data(data_path, name_to_save, mode):
    lines = open(data_path).read().split('\n')
    with open(name_to_save + "_src-" + mode +".txt", "w") as f1, open(name_to_save + "_tgt-" + mode + ".txt", "w") as f2:
        for line in lines[3: len(lines) - 1]:
            input_text, target_text, *c = line.split('\t')
            input_text,_ = input_text.split('>')
            input_text = input_text.split()
            target_text = target_text.split()
            f1.write(" ".join(input_text) + " \n")
            f2.write(" ".join(target_text) + "\n")

In [3]:
data_path = "data/US_patents_1976-Sep2016_1product_reactions_train.csv"
name_to_save = "data/USP"
mode = "train"
separate_data(data_path, name_to_save, mode)

In [4]:
data_path = "data/US_patents_1976-Sep2016_1product_reactions_valid.csv"
name_to_save = "data/USP"
mode = "valid"
separate_data(data_path, name_to_save, mode)

In [5]:
opt = PreprocessorOpts(train_src=name_to_save + "_src-train.txt", 
                  train_tgt=name_to_save + "_tgt-train.txt", 
                  valid_src=name_to_save + "_src-valid.txt",
                  valid_tgt=name_to_save + "_tgt-valid.txt",
                  save_data="data/USP")
src_nfeats = inputters.get_num_features(
    opt.data_type, opt.train_src, 'src')
tgt_nfeats = inputters.get_num_features(
    opt.data_type, opt.train_tgt, 'tgt')
fields = inputters.get_fields(opt.data_type, src_nfeats, tgt_nfeats)
train_dataset_files = build_save_dataset('train', fields, opt)
build_save_dataset('valid', fields, opt)
build_save_vocab(train_dataset_files, fields, opt)

In [6]:
opt = Opts(data="data/USP", save_model="demo_model", train_steps=50, epochs=1)
single_main(opt, -1)

[2018-11-14 23:56:51,514 INFO] Loading train dataset from data\USP.train.0.pt, number of examples: 189902
[2018-11-14 23:56:51,522 INFO]  * vocabulary size. source = 178; target = 101
[2018-11-14 23:56:51,524 INFO] Building model...
[2018-11-14 23:56:51,690 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(178, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(101, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500,